In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
PI0 = ""
CCPI0 = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    PI0 = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    PI0 = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
    
mc = uproot.open(ls.ntuple_path+NU+".root")[fold][tree]
nc = uproot.open(ls.ntuple_path+PI0+".root")[fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

uproot_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

variables = [
    "shr_bkt_pdg", "selected", "nu_pdg",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nu_e","trk_bkt_pdg","ccnc",
    "mc_pdg", "npi0",
    "topological_score","trk_len",
    "evnunhits", "nslice", "selected",
    "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z"]

nue = nue.pandas.df(variables + ["weightSpline"], flatten=False)
mc = mc.pandas.df(variables + ["weightSpline"], flatten=False)
nc = nc.pandas.df(variables + ["weightSpline"], flatten=False)
ccpi0 = ccpi0.pandas.df(variables + ["weightSpline"], flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + ["weightSpline"], flatten=False)
lee = lee.pandas.df(variables + ["weightSpline", "leeweight"], flatten=False)

In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
nc = nc.query('category != 5')
ccpi0 = ccpi0.query('category != 5')

In [ ]:
df_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["topological_score"])
data["bnbdata"] = np.ones_like(data["topological_score"])

In [ ]:
lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11
nc["is_signal"] = nc["category"] == 11
ccpi0["is_signal"] = ccpi0["category"] == 11

lee.loc[lee['category'] == 1, 'category'] = 111
lee.loc[lee['category'] == 10, 'category'] = 111
lee.loc[lee['category'] == 11, 'category'] = 111

In [ ]:
dfbdt_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    "nc": nc,
    "cc": ccpi0,
    "lee": lee
}

scaling = 1
#scaling = 132.0/0.486
#scaling = 132.0/3.9

SPLIT = 1.0

#''' 1115
if ISRUN3:
    weights = {
        "mc": 5.53e-3 * SPLIT * scaling,
        "nue": 1.26e-4 * SPLIT * scaling,
        "ext": 3.51e-2 * SPLIT * scaling,
        "dirt": 2.26e-2 * scaling,
        "lee": 1.26e-4 * SPLIT * scaling,
        "nc": 3.21e-3 * SPLIT * scaling,
        "cc": 1.35e-3 * SPLIT * scaling,
    }
    pot = 0.742e19*scaling
else:
    weights = {
        "mc": 2.82e-2 * SPLIT * scaling,
        "nue": 7e-4 * SPLIT * scaling,
        "ext": 4.42e-1 * SPLIT * scaling,
        "dirt": 1.15e-1 * scaling,
        "lee": 7e-4 * SPLIT * scaling,
        "nc": 1.34e-2 * SPLIT * scaling,
        "cc": 1.06e-2 * SPLIT * scaling,
    }
    pot = 3.71e19*scaling
#'''



my_plotter = plotter.Plotter(samples, weights, pot=pot)

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
#QUERY += ' and topological_score > 0.06'
#QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
#QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
#QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'

print (QUERY)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_x',50,(-5,260),"topological-score"

fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    #kind="interaction",
    #kind="sample",
    #kind='particle_pdg',
    draw_sys=False,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)

#print("Profile likelihood: %.2f sigma @ 1.32e21 POT" % my_plotter.significance_likelihood)
#print("s/sqrt(b): %.2f sigma @ 1.32e21 POT" % my_plotter.significance)

#ax1.set_ylim(0,20)
#ax1.set_yscale("log")
#ax1.set_ylim(0,0.25)
#ax2.set_ylim(0.5,1.5)
fig.savefig(ls.plots_path+VARIABLE+"_"+date_time+".pdf")